# Rock Paper Scissor Game

## Envrionment setup
1. To set up environment, inatall anaconda or miniconda, set up environment using 
`conda env create --file environment.yml`
2. Activate the envrionment 
`conda activate rock_paper_scissors_env`
3. Select python kernel rock_paper_scissors_env
4. Enjoy your game

Please note that hand tracking module is not available for Mac with M1 CPU. An module not found error will be occurred when setting up the environment but 
the code will bypass the hand tracking module and predict your option using the whole capture screen. 

This is the sample code taken from cvznoe GitHub. Not sure the reason but it can make the next cell running properly

In [1]:
try:
    import cvzone
    from cvzone.HandTrackingModule import HandDetector
    include_hand_detection = True
except ImportError:
    include_hand_detection = False

import cv2

if include_hand_detection:
    cap = cv2.VideoCapture(0)
    detector = HandDetector(detectionCon=0.8, maxHands=2)

    count = 10
    while count > 0:
        count -= 1

        # Get image frame
        success, img = cap.read()
        # Find the hand and its landmarks
        hands, img = detector.findHands(img)  # with draw
        # hands = detector.findHands(img, draw=False)  # without draw

        if hands:
            # Hand 1
            hand1 = hands[0]
            lmList1 = hand1["lmList"]  # List of 21 Landmark points
            bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
            centerPoint1 = hand1['center']  # center of the hand cx,cy
            handType1 = hand1["type"]  # Handtype Left or Right

            fingers1 = detector.fingersUp(hand1)

            if len(hands) == 2:
                # Hand 2
                hand2 = hands[1]
                lmList2 = hand2["lmList"]  # List of 21 Landmark points
                bbox2 = hand2["bbox"]  # Bounding box info x,y,w,h
                centerPoint2 = hand2['center']  # center of the hand cx,cy
                handType2 = hand2["type"]  # Hand Type "Left" or "Right"

                fingers2 = detector.fingersUp(hand2)

                # Find Distance between two Landmarks. Could be same hand or different hands
                length, info, img = detector.findDistance(lmList1[8], lmList2[8], img)  # with draw
                # length, info = detector.findDistance(lmList1[8], lmList2[8])  # with draw
        # Display
        cv2.imshow("Image", img)
        cv2.waitKey(1)
        
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
from keras.models import load_model
import numpy as np
from datetime import datetime, timedelta

USER_OPTIONS_EXPIRY_TIME = 3
MODEL_PREDICTION_MIN_INTERVAL = 0.7
ENABLE_HAND_DETECTION = True

class Options:
    ROCK = 0
    PAPER = 1
    SCISSOR = 2
    NONE = 3

class Result:
    TIE = 0
    USER_WINS = 1
    COMPUTER_WINS = 2

winning_text_map = {
    Result.TIE : "Tie",
    Result.USER_WINS : "User Wins",
    Result.COMPUTER_WINS : "Computer Wins"
}

def getExpiryTime(second=USER_OPTIONS_EXPIRY_TIME):
    return datetime.now() + timedelta(seconds=second)

def checkOption(user_option, computer_option):
    if user_option == computer_option:
        return Result.TIE
    elif user_option > computer_option:
        if computer_option == Options.ROCK and user_option == Options.SCISSOR:
            return Result.COMPUTER_WINS
        else:
            return Result.USER_WINS
    else:
        if user_option == Options.ROCK and computer_option == Options.SCISSOR:
            return Result.USER_WINS
        else:
            return Result.COMPUTER_WINS

def initializeVideoCap():
    cap = cv2.VideoCapture(0)
    detector = HandDetector(detectionCon=0.8, maxHands=2)

    count = 100

    while count > 0:
        # Get image frame
        success, img = cap.read()
        # Find the hand and its landmarks
        hands, img = detector.findHands(img)  # with draw
        # hands = detector.findHands(img, draw=False)  # without draw

        cv2.imshow("Image", img)
        cv2.waitKey(1)

        count -= 1
        
    cap.release()
    cv2.destroyAllWindows()
    # cv2.waitKey(1000)

def startGame():
    model = load_model('./model/keras_model.h5', compile=False)
    
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX

    with open('./model/labels.txt') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        labels.append(line.split(" ")[1].replace("\n", ""))

    computer_class = np.random.randint(Options.ROCK, Options.NONE)
    computer_label = labels[computer_class]

    prev_choice = Options.NONE # initial choice = None
    expiry_time = None
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    is_playing = True

    if include_hand_detection:
        detector = HandDetector(detectionCon=0.8, maxHands=1)

    user_wins = 0
    computer_wins = 0

    message_first_line = 'Welcome to Rock Paper Scissors!'
    message_second_line = f'Hold your options for {USER_OPTIONS_EXPIRY_TIME} seconds'

    print("Game start " + ("with" if include_hand_detection else "without") + " hand recognizer.")

    while True:
        ret, frame = cap.read()
        cv2.putText(frame,
                    message_first_line, 
                    (20, 50), 
                    font, 1, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_4)

        cv2.putText(frame,
                    message_second_line, 
                    (20, 100), 
                    font, 1, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_4)

        tips_message = ""

        if is_playing:
            if prev_choice == Options.NONE:
                tips_message = "Try different angle if the system cannot detects your choice."
            else:
                tips_message = f"Are you showing {predict_label}?"

        cv2.putText(frame,
                    tips_message, 
                    (20, frame.shape[0]-40), 
                    font, 1, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_4)

        if is_playing == True:
            if include_hand_detection:
                img = detector.findHands(frame, draw=False)

                if img and len(img) == 1:
                    bbox = img[0]["bbox"]
                    print(f"hand located [{bbox}]")

                    startX = max(0, bbox[0] - 80)
                    endX = min(bbox[0] + bbox[2] + 80, frame.shape[1])

                    startY = max(0, bbox[1] - 80)
                    endY = min(bbox[1] + bbox[3] + 80, frame.shape[0])

                    frame_hand = frame[startY:endY, startX:endX, :]
                else:
                    frame_hand = None

            else:
                frame_hand = frame

            if frame_hand is not None and frame_hand.shape[0] > 0 and frame_hand.shape[1] > 0:
                print(f"Predict model using frame shape {frame_hand.shape}")
                
                if include_hand_detection:
                    cvzone.cornerRect(frame, (startX, startY, endX-startX, endY-startY))

                resized_frame = cv2.resize(frame_hand, (224, 224), interpolation = cv2.INTER_AREA)
                image_np = np.array(resized_frame)
                normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
                data[0] = normalized_image
                prediction = model.predict(data)
                # print(prediction)

                prediction = np.squeeze(prediction)
                max_option = np.argmax(prediction)

                predict_class = max_option if prediction[max_option] >= MODEL_PREDICTION_MIN_INTERVAL else Options.NONE
                predict_label = labels[predict_class]

                print(predict_label)

                if predict_class < Options.NONE:
                    # user chooses another options (rock, paper or scissor), update previous choice and expiry date
                    if prev_choice != predict_class:
                        prev_choice = predict_class
                        expiry_time = getExpiryTime()

                        print(f"User select {predict_label}, update expiry time {expiry_time}")

                    # user continue previous option, check whether he/she has hold the same option for while
                    elif datetime.now() > expiry_time:
                        # finish this round
                        winning = checkOption(predict_class, computer_class)
                        message_first_line = f'Computer chooses {computer_label}, you chooses {predict_label}, {winning_text_map[winning]}'

                        if winning == Result.COMPUTER_WINS:
                            computer_wins += 1
                        elif winning == Result.USER_WINS:
                            user_wins += 1

                        message_first_line += f" - {user_wins} : {computer_wins}"
                        is_playing = False

                        # if either users or computer has won 3 times, finish this game
                        if computer_wins == 3 or user_wins == 3:
                            message_second_line = "Congratulations! " if user_wins == 3 else ""
                            message_second_line += "Press 'r' to restart the game or press 'q' to quit the game"
                        else:
                            message_second_line = f"Click 'c' to continue"

                # new user option is none, clear previous choice and expiry time
                else:
                    prev_choice = Options.NONE
                    expiry_time = None

            # hand not found, clear previous choice and expiry time
            else:
                prev_choice = Options.NONE
                expiry_time = None

        cv2.imshow('Image', frame)
        key = cv2.waitKey(10) & 0xFF

        # Press q to close the window
        if key == ord("q"):
            break

        if is_playing == False:
            # Press r to restart the game
            if key == ord("r"):
                is_playing = True
                computer_class = np.random.randint(Options.ROCK, Options.NONE)
                computer_label = labels[computer_class]

                user_wins = 0
                computer_wins = 0

                prev_choice = Options.NONE # initial choice = None
                expiry_time = None

                message_first_line = 'Welcome to Rock Paper Scissors!'
                message_second_line = f'Hold your options for {USER_OPTIONS_EXPIRY_TIME} seconds'

            # press c to continue
            elif key == ord("c"):
                is_playing = True

                # next round
                prev_choice = Options.NONE
                expiry_time = None

                computer_class = np.random.randint(Options.ROCK, Options.NONE)
                computer_label = labels[computer_class]

                message_second_line = f'Hold your options for {USER_OPTIONS_EXPIRY_TIME} seconds'

    # After the loop release the cap object
    cap.release()
    cv2.waitKey(1)
    # Destroy all the windows
    cv2.destroyAllWindows()
    cv2.waitKey(1)

startGame()

2022-02-07 21:13:25.420469: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Game start with hand recognizer.
